**Binomial option pricing**

Binomial option pricing is a mathematical model used to evaluate prices of options. This model provides a flexible and straightforward way to account for the fluctuating nature of the underlying asset's price over time, making it possible to estimate the price of options in a risk-neutral world.

The binomial model uses a discrete-time framework, dividing the time until the option's expiration into a series of intervals. At each interval, the model assumes that the price of the underlying asset can move to one of two possible
values: up or down. This creates a binomial tree of possible asset prices, where each node represents a possible price of the asset at a given point in time.

The key equations in the binomial option pricing model include:

1. **Price Up and Down Movements:**
   - The price moves up to $u \times S$ or down to $d \times S$, where $S$ is the current price, $u$ is the multiplier for an up move, and $d$ is the multiplier for a down move.

2. **Risk-neutral Probability ($q$):**
   - The probability of an up move in a risk-neutral world is calculated as:     $$ q = \frac{e^{r \Delta t} - d}{u - d} $$
   where $r$ is the risk-free interest rate, $\Delta t$ is the length of each time interval, and $e$ is the base of the natural logarithm. The probability of a down move is $1 - q$.

3. **Option Value at Expiration:**
   - For a call option (the right to buy), the value at expiration is:
     $$\max(S_T - K, 0) $$
   - For a put option (the right to sell), the value at expiration is:     $$\max(K - S_T, 0) $$
   where $S_T$ is the price of the underlying asset at expiration, and $K$ is the strike price of the option.

4. **Option Value Before Expiration:**
   - The value of the option at any node before expiration is calculated using the risk-neutral probabilities:
     $$V = e^{-r \Delta t} (qV_{\text{up}} + (1 - q)V_{\text{down}})$$

   where $V_{\text{up}}$ and $V_{\text{down}}$ are the values of the option in the next time step for the up and down price movements, respectively.

By applying these equations iteratively from the expiration date back to the present, the binomial model provides the present value of an option based on the expected future price movements of the underlying asset, adjusted for the time value of money. This model is especially useful for American options, which can be exercised at any time before expiration, as it easily accommodates the option's changing value over time.


In [4]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt


In [5]:
def binomial_coefficient(n, k):
    """
    Calculates the binomial coefficient, also known as the combination or "n choose k".
    It represents the number of ways to choose k items from n without regard to order.

    Parameters:
    n (int): Total number of items.
    k (int): Number of items to choose.

    Returns:
    int: The binomial coefficient of n and k.
    """
    return np.math.factorial(n) / (np.math.factorial(n-k) * np.math.factorial(k))



In [6]:

def binomial_option_pricing(initial_stock_price, strike_price, maturity, annual_interest_rate, volatility, periods):
    """
    Calculates the price of a European call option using the Binomial Option Pricing model.

    Parameters:
    initial_stock_price (float): The initial price of the underlying stock.
    strike_price (float): The strike price of the option.
    maturity (float): The time to maturity of the option, in years.
    annual_interest_rate (float): The risk-free annual interest rate.
    volatility (float): The volatility of the underlying stock.
    periods (int): The number of periods for the binomial tree.

    Returns:
    float: The calculated price of the call option.
    """
    delta_t = maturity / periods
    up_factor = np.exp(volatility * np.sqrt(delta_t))
    down_factor = np.exp(-volatility * np.sqrt(delta_t))
    probability = (np.exp(annual_interest_rate * delta_t) - down_factor) / (up_factor - down_factor)
    call_price = 0

    for k in range(periods + 1):
        probability_k = binomial_coefficient(periods, k) * probability**k * (1-probability)**(periods-k)
        stock_price_k = initial_stock_price * (up_factor)**(2*k - periods)
        call_price += max(stock_price_k - strike_price, 0) * probability_k

    return call_price * np.exp(-annual_interest_rate * maturity)

# Parameters for the model
initial_stock_price = 50  # Initial stock price (S0)
strike_price = 60         # Strike price (K)
maturity = 1               # Time to maturity in years (T)
annual_interest_rate = 0.06  # Annual interest rate (r)
volatility = 0.2           # Stock volatility (sigma)
periods = 60               # Number of periods (N)

# Calculate option prices
call_option_price = binomial_option_pricing(initial_stock_price, strike_price, maturity, annual_interest_rate, volatility, periods)
put_option_price = strike_price * np.exp(-annual_interest_rate * maturity) + call_option_price - initial_stock_price  # Put price using put-call parity

print(f'Call option price: {call_option_price:.5f}')
print(f'Put option price:  {put_option_price:.5f}')


Call option price: 1.76131
Put option price:  8.26718


<ipython-input-5-e169b794d32f>:13: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return np.math.factorial(n) / (np.math.factorial(n-k) * np.math.factorial(k))
